In [4]:
#These are the libraries you can use.  You may add any libraries directy related to threading if this is a direction
#you wish to go (this is not from the course, so it's entirely on you if you wish to use threading).  Any
#further libraries you wish to use you must email me, james@uwaterloo.ca, for permission.

%pip install numpy-financial yfinance pandas numpy matplotlib --quiet
from IPython.display import display, Math, Latex

import pandas as pd
import numpy as np
import numpy_financial as npf
import yfinance as yf
import matplotlib.pyplot as plt
import random
from datetime import datetime




Note: you may need to restart the kernel to use updated packages.


## Group Assignment
### Team Number: 18
### Team Member Names: Poneesh Kumar, Ian Leung, Vivaan Goel
### Team Strategy Chosen: Market Beat(Market Beat, Market Meet, Risk-Free)

In [5]:
filename = "Tickers_Example.csv"
tickers = pd.read_csv(filename, header=None, names=['Ticker'])
tickers.head()

,Ticker
0,AAPL
1,ABBV
2,ABT
3,ACN
4,AGN


## Contribution Declaration

The following team members made a meaningful contribution to this assignment:

Poneesh, Ian, Vivaan

In [6]:

# temp
# beta, volume, regularMarketVolume, averageVolume, average(Daily)Volume10days/Day, marketCap, 
# currency, earningsQuarterlyGrowth, financialCurrency, market = us_market

history = {}
def averageDailyVolume(ticker):
    start_date = "2024-10-01"
    end_date = "2025-09-30"

    history[ticker.ticker] = ticker.history(start=start_date, end=end_date)

    daily_volume = history[ticker.ticker]['Volume']

    daily_volume.index = daily_volume.index.tz_localize(None)

    volume_df = daily_volume.to_frame()
    volume_df['Months'] = volume_df.index.to_period('M')

    daily_volume.index = pd.to_datetime(daily_volume.index)
    daily_volume.index = daily_volume.index.strftime('%Y-%m-%d') 
    volume_df.index = pd.to_datetime(volume_df.index)
    volume_df.index = volume_df.index.strftime('%Y-%m-%d') 

    monthly_counts = volume_df.groupby('Months')['Volume'].count()
    valid_months = monthly_counts[monthly_counts >= 18].index
    valid_vol_df = volume_df[volume_df['Months'].isin(valid_months)]

    volumes = []
    
    for volume in valid_vol_df['Volume']:
        volumes.append(volume)
    
    average_daily_vol = np.mean(volumes)
    return average_daily_vol
    



In [28]:
def scoring1(ticker):
    start_date = "2025-11-01"
    end_date = "2025-11-14"

    history[ticker.ticker] = ticker.history(start=start_date, end=end_date)

    daily_close = history[ticker.ticker]['Close']
    daily_volume = history[ticker.ticker]['Volume']

    daily_close.index = daily_close.index.tz_localize(None)
    daily_volume.index = daily_volume.index.tz_localize(None)

    daily_close.index = pd.to_datetime(daily_close.index)
    daily_close.index = daily_close.index.strftime('%Y-%m-%d') 
    daily_volume.index = pd.to_datetime(daily_volume.index)
    daily_volume.index = daily_volume.index.strftime('%Y-%m-%d') 
    
    price_change = daily_close.pct_change()
    price_change.drop(index=price_change.index[0], inplace=True)
    volume_change = daily_volume.pct_change()
    volume_change.drop(index=volume_change.index[0], inplace=True)
    
    avg_price_roc = np.mean(price_change)*100
    avg_vol_roc = np.mean(volume_change)*100

    # Calculating the Relative Strength Index
    delta = daily_close.astype(float).diff()

    gain = delta.clip(lower=0)
    loss = -delta.clip(upper=0)

    window = 7
    avg_gain = gain.rolling(window=window, min_periods=window).mean()
    avg_loss = loss.rolling(window=window, min_periods=window).mean()

    rs = avg_gain / avg_loss
    rsi = 100 - (100 / (1 + rs))
    latest_rsi = rsi.iloc[-1]

    # Calculating the simple moving average
    sma_7 = daily_close.astype(float).rolling(window=7).mean().iloc[-1]
    

    return avg_price_roc, avg_vol_roc, latest_rsi, sma_7

In [29]:
tickers_df = pd.DataFrame(columns=[
    'Ticker', 'Sector', 'Market Cap', 'Country', 'Market',
    'Avg Daily Vol', 'Beta',
    'Price % ROC', 'Vol % ROC',
    'RSI', 'SMA'
])

row = 0
for ticker_sym in tickers['Ticker']:
    ticker = yf.Ticker(ticker_sym)
    try:
        price_roc, vol_roc, rsi_val, sma_val = scoring1(ticker)
        tickers_df.loc[row] = {
        'Ticker': ticker_sym,
        'Sector': ticker.info['sector'],
        'Market Cap': ticker.info['marketCap'],
        'Country': ticker.info['country'],
        'Market': ticker.info['market'],
        'Avg Daily Vol': averageDailyVolume(ticker),
        'Beta': ticker.info['beta'],
        'Price % ROC': price_roc,
        'Vol % ROC': vol_roc,
        'RSI': rsi_val,
        'SMA': sma_val
        }

        row+= 1
    except:
        print(f"{ticker_sym} invalid, skipping...")

tickers_df

$AGN: possibly delisted; no timezone found


AGN invalid, skipping...


$CELG: possibly delisted; no timezone found


CELG invalid, skipping...


$MON: possibly delisted; no timezone found


MON invalid, skipping...


$RTN: possibly delisted; no timezone found


RTN invalid, skipping...


,Ticker,Sector,Market Cap,Country,Market,Avg Daily Vol,Beta,Price % ROC,Vol % ROC,RSI,SMA
0,AAPL,Technology,3985534877696,United States,us_market,5.384430e+07,1.109,0.195537,0.370499,64.275530,271.242445
1,ABBV,Healthcare,411641544704,United States,us_market,6.362559e+06,0.352,1.161068,3.613915,92.752868,223.481428
2,ABT,Healthcare,219558903808,United States,us_market,6.189108e+06,0.719,0.540395,-0.376179,74.999945,126.688571
3,ACN,Technology,150157328384,Ireland,us_market,3.444719e+06,1.262,-0.029409,0.248503,59.717036,245.161429
4,AIG,Financial Services,41965789184,United States,us_market,4.416646e+06,0.622,-0.069889,5.382320,35.359793,76.992857
5,AMZN,Consumer Cyclical,2380604112896,United States,us_market,4.246964e+07,1.368,-0.818906,-5.634273,27.088211,245.275713
6,AXP,Financial Services,239828860928,United States,us_market,2.800084e+06,1.147,0.112333,4.784922,59.733776,367.944288
7,BA,Industrials,141149732864,United States,us_market,9.557990e+06,1.170,-0.617167,-0.258184,16.952369,195.547143
8,BAC,Financial Services,385309409280,United States,us_market,3.933537e+07,1.304,-0.153816,8.267657,41.966407,53.281429
9,BB.TO,Technology,3531513344,Canada,ca_market,2.494208e+06,1.023,-1.241720,0.665797,24.444431,6.404286


In [9]:
filtered_df = tickers_df[(tickers_df["Country"] == 'Canada') | (tickers_df["Country"] == "United States")].reset_index(drop=True)
filtered_df = filtered_df[(filtered_df["Market"] == 'ca_market') | (filtered_df["Market"] == "us_market")].reset_index(drop=True)
filtered_df = filtered_df[filtered_df["Avg Daily Vol"] > 5000].reset_index(drop=True)
filtered_df

,Ticker,Sector,Market Cap,Country,Market,Avg Daily Vol,Beta,Price % ROC,Vol % ROC
0,AAPL,Technology,3985534877696,United States,us_market,5.384430e+07,1.109,0.195537,0.370499
1,ABBV,Healthcare,411641544704,United States,us_market,6.362559e+06,0.352,1.161068,3.613915
2,ABT,Healthcare,219558903808,United States,us_market,6.189108e+06,0.719,0.540395,-0.376179
3,AIG,Financial Services,41965789184,United States,us_market,4.416646e+06,0.622,-0.069889,5.382320
4,AMZN,Consumer Cyclical,2380604112896,United States,us_market,4.246964e+07,1.368,-0.818906,-5.634273
5,AXP,Financial Services,239828860928,United States,us_market,2.800084e+06,1.147,0.112333,4.784922
6,BA,Industrials,141149732864,United States,us_market,9.557990e+06,1.170,-0.617167,-0.258184
7,BAC,Financial Services,385309409280,United States,us_market,3.933537e+07,1.304,-0.153816,8.267657
8,BB.TO,Technology,3531513344,Canada,ca_market,2.494208e+06,1.023,-1.241720,0.665797
9,BIIB,Healthcare,24635711488,United States,us_market,1.555065e+06,0.145,1.066874,7.368712


In [29]:
# print(history)
print(history['AAPL'])
# yf.Ticker('AAPL').info.keys()

                                 Open        High         Low       Close  \
Date                                                                        
2024-10-01 00:00:00-04:00  228.235504  228.364766  222.487853  224.944031   
2024-10-02 00:00:00-04:00  224.625815  226.097528  221.771882  225.510834   
2024-10-03 00:00:00-04:00  223.880011  225.540663  222.070204  224.407043   
2024-10-04 00:00:00-04:00  226.624566  226.724012  222.875676  225.530731   
2024-10-07 00:00:00-04:00  223.243613  224.426955  220.091355  220.449341   
...                               ...         ...         ...         ...   
2025-09-23 00:00:00-04:00  255.632202  257.090779  253.334426  254.183594   
2025-09-24 00:00:00-04:00  254.972829  255.492329  250.796869  252.065643   
2025-09-25 00:00:00-04:00  252.964772  256.920944  251.466225  256.621216   
2025-09-26 00:00:00-04:00  253.853917  257.350528  253.534220  255.212601   
2025-09-29 00:00:00-04:00  254.313473  254.753049  252.764971  254.183594   